In [ ]:
#@title 🏥 Pharmacy Start Address

#!pip install openrouteservice  -q
# https://account.heigit.org/manage/key
# https://openrouteservice-py.readthedocs.io/en/latest/

import ipywidgets as widgets
from IPython.display import display, clear_output

import requests
import time

import openrouteservice
from openrouteservice.optimization import Vehicle, Job
import urllib.parse

import folium
from folium import plugins
from IPython.display import display, HTML

api_key = "5b3ce3597851110001cf624872f25c4e63de429880fd7c7e9e1727b2"

# --- Cell 1: Control Panel ---

# PHARMACY OPTIONS
pharmacy_options = {
    "University Pl Pharmacy": "23 University Pl Blvd, Jersey City, NJ 07305",
    "MediSave Pharmacy": "362 Monroe St C, Passaic, NJ 07055",
    "Other (enter manually below)": ""
}

pharmacy_dropdown = widgets.Dropdown(
    options=list(pharmacy_options.keys()),
    value="University Pl Pharmacy",
    description="🏥 Pharmacy:"
)

pharmacy_manual_input = widgets.Text(
    placeholder="Enter custom pharmacy address...",
    description="✍️ Custom:",
    layout=widgets.Layout(width='80%')
)

# NUMBER OF ADDRESSES
num_fields_input = widgets.BoundedIntText(
    value=20,
    min=1,
    max=50,
    step=1,
    description="🧾 # Addresses:",
    layout=widgets.Layout(width='250px')
)

# NUMBER OF DRIVERS
num_drivers_input = widgets.BoundedIntText(
    value=2,
    min=1,
    max=10,
    step=1,
    description="🚚 # Drivers:",
    layout=widgets.Layout(width='250px')
)




time_now_input = widgets.Text(
    placeholder="10:00",
    description="✍️ time_now_input:",
    layout=widgets.Layout(width='250px')
)

end_of_day = widgets.Text(
    placeholder="18:00",
    description="✍️ end_of_day:",
    layout=widgets.Layout(width='250px')
)

Facility_delivery_time = widgets.Text(
    placeholder="13:30",
    description="✍️ Facility_delivery_time:",
    layout=widgets.Layout(width='250px')
)

# DISPLAY MAIN CONFIG
display(pharmacy_dropdown, pharmacy_manual_input, num_fields_input, num_drivers_input, time_now_input, end_of_day, Facility_delivery_time)

print("✅✅✅✅✅✅✅")


In [ ]:
#@title 📦 Delivery Addresses & Driver Constraints

# --- Output Areas ---
addresses_output = widgets.Output()
drivers_output = widgets.Output()
display(addresses_output, drivers_output)

# --- Storage ---
address_widgets = []
priority_widgets = []
driver_widgets = []
urgency_widgets = []
time_window_widgets = []

# --- Helper ---
def get_pharmacy_address():
    selected = pharmacy_dropdown.value
    return pharmacy_manual_input.value.strip() if selected == "Other (enter manually below)" else pharmacy_options[selected]

# --- Create Address Fields ---
# --- Create Address Fields ---
def create_address_fields(num_fields):
    address_widgets.clear()
    priority_widgets.clear()
    urgency_widgets.clear()
    time_window_widgets.clear()
    addresses_output.clear_output(wait=True)

    with addresses_output:
        print(f"📦 Delivery Addresses (Start at: {get_pharmacy_address()}):")
        rows = []
        for i in range(num_fields):
            addr = widgets.Text(
                placeholder="Enter address",
                description=f"Address {i+1}:",
                layout=widgets.Layout(width='35%')
            )

            prio = widgets.Dropdown(
                options=[str(p) for p in range(1, 10)],
                value='5',
                description="Priority",
                layout=widgets.Layout(width='130px')
            )

            urgent = widgets.Checkbox(
                value=False,
                description="Urgent",
                indent=False,
                layout=widgets.Layout(width='100px')
            )

            time_window = widgets.Text(
                placeholder="e.g., 14:00-16:30",
                description="Time",
                layout=widgets.Layout(width='180px')
            )

            address_widgets.append(addr)
            priority_widgets.append(prio)
            urgency_widgets.append(urgent)
            time_window_widgets.append(time_window)

            rows.append(widgets.HBox([addr, prio, urgent, time_window]))

        # Batch display in chunks (e.g., 5 at a time)
        batch_size = 5
        display(widgets.VBox([widgets.VBox(rows[i:i + batch_size]) for i in range(0, len(rows), batch_size)]))

# --- Create Driver Fields ---
def create_driver_fields(num_drivers, default_max_tasks):
    driver_widgets.clear()
    drivers_output.clear_output(wait=True)

    with drivers_output:
        print("🚚 Driver Constraints:")
        rows = []
        for i in range(num_drivers):
            name = widgets.Text(
                value=f"Driver{i+1}",
                description="Name",
                layout=widgets.Layout(width='250px')
            )
            max_tasks = widgets.BoundedIntText(
                value=default_max_tasks,
                min=1,
                max=100,
                description="Max Tasks",
                layout=widgets.Layout(width='220px')
            )
            roundtrip = widgets.Checkbox(value=False, description="Round Trip")
            home = widgets.Text(
                placeholder="End address (if not round trip)",
                description="Home",
                layout=widgets.Layout(width='40%')
            )
            driver_widgets.append((name, max_tasks, roundtrip, home))
            rows.append(widgets.HBox([name, max_tasks, roundtrip, home]))

        display(widgets.VBox(rows))

# --- Callback ---
def refresh_fields(*args):
    create_address_fields(num_fields_input.value)
    create_driver_fields(num_drivers_input.value, num_fields_input.value)

# --- Observe Triggers (run only once) ---
num_fields_input.unobserve_all()
num_drivers_input.unobserve_all()
#pharmacy_dropdown.unobserve_all()
#pharmacy_manual_input.unobserve_all()

num_fields_input.observe(refresh_fields, names='value')
num_drivers_input.observe(refresh_fields, names='value')
pharmacy_dropdown.observe(refresh_fields, names='value')
pharmacy_manual_input.observe(refresh_fields, names='value')

# --- Initial Render ---
refresh_fields()


In [ ]:
#@title 📦 Results


# ── requirements ────────────────────────────────────────────────
# pip install openrouteservice scikit-learn python-dotenv

from openrouteservice import Client
from openrouteservice.optimization import Job, Vehicle
from sklearn.cluster import KMeans
import datetime as dt
import urllib.parse


import requests
import time
from typing import Dict, Tuple

# Global cache: address → (lon, lat)
_geocode_cache: Dict[str, Tuple[float, float]] = {}



# ── helper ──────────────────────────────────────────────────────
def _sec_today(hh_mm: str) -> int:
    """convert 'HH:MM' to seconds since midnight"""
    h, m = map(int, hh_mm.split(":"))
    return h * 3600 + m * 60

def geocode_free(client: Client, address: str): # geocode_addr
    """lightweight wrapper around ORS Pelias search → [lon, lat]"""
    if address in _geocode_cache:
        return _geocode_cache[address]

    rsp = client.pelias_search(text=address)
    if not rsp["features"]:
        raise ValueError(f"Could not geocode: {address}")
    coords = rsp["features"][0]["geometry"]["coordinates"]
    _geocode_cache[address] = coords  # store in cache
    return coords



def geocode_addr(ors, address: str) -> Tuple[float, float]:
    """Geocode using a dict cache + maps.co fallback."""
    # Return cached result if available
    if address in _geocode_cache:
        return _geocode_cache[address]

    # Not in cache — perform API lookup
    time.sleep(1)  # respect rate limit
    url = "https://geocode.maps.co/search"  # https://geocode.maps.co/join/
    params = {
        "q": address,
        "api_key": "6861820819cc3625503686uwk18b7ea"  ## https://geocode.maps.co
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if not data:
        raise ValueError(f"Could not geocode: {address}")

    coords = (float(data[0]["lon"]), float(data[0]["lat"]))
    _geocode_cache[address] = coords  # store in cache
    return coords



# ── a. seconds ➜ "HH:MM" ─────────────────────────────────────────
def hhmm_from_sec(sec: int) -> str:
    sec %= 24 * 3600                       # keep within one day
    return f"{sec//3600:02d}:{sec%3600//60:02d}"

# ── b. lon/lat ➜ human-readable address ─────────────────────────
def reverse_geocode_addr(client: Client, lon: float, lat: float) -> str:
    rsp = client.pelias_reverse(point=[lon, lat], size=1)  # ORS Pelias
    feats = rsp.get("features", [])
    return feats[0]["properties"]["label"] if feats else "unknown"




# ── main optimiser ──────────────────────────────────────────────
def optimize_pharmacy_routes(
    api_key: str,
    pharmacy_address: str,
    patient_addresses: list[str],
    urgency_flags: list[bool],
    target_arrival_time: list[str],
    driver_home_addresses: list[str],
    now: str = "10:00",
    end_day:  str = "18:00",
    urgent_due:  str = "13:30", ):
    """
    Returns ORS optimisation result & prints a readable summary.
    """
    if len(patient_addresses) != len(urgency_flags):
        raise ValueError("patient_addresses and urgency_flags must align")

    n_drivers = len(driver_home_addresses)
    now_s      = _sec_today(now)
    end_day_s  = _sec_today(end_day)           # drivers off-duty at 6 pm
    urgent_due_s = _sec_today(urgent_due)           # facilities close 2 pm
    target_arrival_time_s = _sec_today(target_arrival_time)



    # ── 1.  set up ORS client & geocode everything ──────────────
    ors = Client(key=api_key, timeout=180)
    _geocode_cache[pharmacy_address] = geocode_addr(ors, pharmacy_address)
    for addr in patient_addresses + driver_home_addresses:
        _geocode_cache.setdefault(addr, geocode_addr(ors, addr))




    # ── 2.  build K-means clusters to split work fairly ─────────
    pts = [_geocode_cache[a] for a in patient_addresses]
    kmeans = KMeans(n_clusters=n_drivers, n_init="auto").fit(pts)
    cluster_ids = kmeans.labels_            # 0…n_drivers-1

    # ── 3.  build Jobs list (one per patient) ───────────────────
    jobs = []


    for j, (addr, urgent) in enumerate(zip(patient_addresses, urgency_flags)):
        cid = int(cluster_ids[j])
        tw  = [now_s, urgent_due_s] if urgent else [now_s, end_day_s]
        jobs.append(
            Job(
                id=j,
                location=_geocode_cache[addr],
                skills=[cid],              # forces cluster → driver
                priority=10 if urgent else 1,
                service=900,               # constant 15 min hand-off
                amount=[1],
                time_windows=[tw],))




    # ── 4.  build Vehicle list (one per driver) ─────────────────
    vehicles = []


    for i, drivr in enumerate(driver_settings, start=1):
        if drivr["round_trip"]:
            vehicles.append(Vehicle(
                id=i,
                profile='driving-car',
                start=_geocode_cache[pharmacy_address],
                end=_geocode_cache[pharmacy_address],
                skills  = [vid],           # matches cluster id
                capacity=[drivr["max_tasks"]],
                time_window=[now_s, end_day_s],

            ))
        else:
            if drivr["home"]:
                vehicles.append(Vehicle(
                    id=i,
                    profile='driving-car',
                    start=_geocode_cache[pharmacy_address],
                    end=_geocode_cache[drivr["home"]],
                    skills  = [vid],           # matches cluster id
                    capacity=[drivr["max_tasks"]]
                ))
            else:
                vehicles.append(Vehicle(
                    id=i,
                    profile='driving-car',
                    start=_geocode_cache[pharmacy_address],
                    skills  = [vid],           # matches cluster id

                    capacity=[drivr["max_tasks"]]
                ))



    # ── 5.  run optimisation ───────────────────────────────────
    result = ors.optimization(
        jobs      = jobs,
        vehicles  = vehicles,
        geometry  = True,          # polyline for later mapping
    )


    return result







# collecting data from the form to pass into the optimization function

driver_settings = []
for name, max_tasks, round_trip, home in driver_widgets:
        driver_settings.append({
            "name": name.value.strip(),
            "max_tasks": int(max_tasks.value),
            "round_trip": round_trip.value,
            "home": home.value.strip()
        })


# function call and printing :

result =  optimize_pharmacy_routes(
    api_key,
    pharmacy_address,
    patient_addresses,
    urgency_flags,
    target_arrival_time,
    driver_settings,
    now = "10:00",
    end_day= "18:00",
    urgent_due= "13:30" )







# ── 6.  pretty-print plan  ──────────────────────────────────


print("\n=== DAILY PLAN ===")
for route in result["routes"]:
        vehicle_id   = route["vehicle"]
        dur   = route["duration"] / 60     # minutes
        dist  = route["distance"] / 1609.34   # mile
        stops = [step["id"] for step in route["steps"] if step["type"]=="job"]
        print(f"\nDriver {vehicle_id+1}:  {len(stops)} stops | "
              f"{dur:.1f} min | {dist:.2f} miles")
        #for s in route["steps"]:
        #    if s["type"] == "job":
        #        addr = patient_addresses[s["id"]]
        #        time_of_arrival =hhmm_from_sec(s["arrival"])
        #        tag  = "URGENT" if urgency_flags[s["id"]] else ""
#


    ##return result
        # Step 5: Parse and generate route for each vehicle
all_routes = []
for route in result['routes']:
        vehicle_id = route['vehicle']
        driver_name = driver_settings[vehicle_id - 1]["name"]
        steps = route['steps']
        ordered_addresses = [pharmacy_address]  # start
        for step in steps:
            if 'job' in step:
                addr = patient_addresses[step["id"]]
                time_of_arrival =hhmm_from_sec(step["arrival"])
                tag  = "URGENT" if urgency_flags[s["id"]] else ""
                #print(step["type"])
                #print(hhmm_from_sec(step["arrival"]))
                #print(reverse_geocode_addr(client, step["location"][0],step["location"][1]))

                print(f"  • {addr} {time_of_arrival} {tag}")
                ordered_addresses.append(patient_addresses[step['job']])
            elif step["type"] == "end":
                end_station = [driver_home_addresses[vehicle_id]]
                print("====end of trip=====")

        #if round_trip:
        #    ordered_addresses.append(patient_addresses[0])  # append return

        # Create Google Maps link
        encoded = [urllib.parse.quote_plus(a) for a in ordered_addresses]
        maps_url = "https://www.google.com/maps/dir/" + "/".join(encoded)
        total_time = round(route['duration'] / 60, 1)  # minutes
        all_routes.append({
            "vehicle_id": vehicle_id,
                        "driver_name": driver_name,

            "addresses": ordered_addresses,
            "google_maps_link": maps_url,
            "total_time": total_time
        })



for route in all_routes:
            print(f"\n🚚 Driver #{route['vehicle_id']} - {route['driver_name']}")
            for stop in route['addresses']:
                print("  📍", stop)
            print("🔗 Google Maps:", route['google_maps_link'])
            print(f"⏱️ Estimated Time: {route['total_minutes']} minutes")
            print("-" * 60)
